# Stage 4b: The Sun (Coronagraph)

A very important part of our Solar System is the Sun itself. It is for this reason that we, as its residents, have many observatories, both ground and space-based, that take measurements related to it and its influence.



1.   Installing the relevant modules
2.   Fetching Images from current spacecraft
3.   Plotting them
4.   Making a video in Python (intro)

## Installing the relevant modules

The first step here will be to get a module installed inside the jupyter notebook. To do that, just **run the following line the first time you open this notebook:**

In [ ]:
!pip install -q sunpy[all] astropy  # This package is used for all things Sun-related, hosted in github: https://github.com/sunpy/sunpy


**After the installation is finished, click on Runtime and Restart the Runtime with Control + M!**

In [ ]:
from sunpy.net import Fido, attrs as a
from astropy import units as u  

from datetime import datetime   # Python's small Time module

Inst = 'LASCO'  
Detect = 'C3' # Here we will use the LASCO coronalgraph
# It is onboard the SOHO mission. 
#Provides us with images of the Sun almost 24-7!


# Now choose the period we would like to get our data from. Please don't do too much at once!

start_time = datetime(2012,8,31,18,0)  # Year,Month,Day,Hour,Minute
end_time = datetime(2012,8,31,23,59)

samplerate_hours = 6 * u.minute  # We need to give our images a cadence for collection. 
# Please keep in mind that aia takes an image in all of its wavelengths every 10 seconds or so.
# Choose a cadence depending on what you want to bring out!

  
#fido = Fido.search(a.Time(start=start_time, end=end_time), a.Instrument(Inst), a.vso.Sample(samplerate_hours))

fido = Fido.search(a.Time(start=start_time, end=end_time), a.Instrument(Inst), a. Detector(Detect), a.Sample(samplerate_hours))

print(fido)

Now, we should have a series of results showing above this text-box. These are available records that we may now download:

In [ ]:
from os import makedirs as newdir

dir_images = 'LASCO_data'
newdir(dir_images, exist_ok = True)

Fido.fetch(fido, path = 'LASCO_data/')

##Looking at our newly acquired data

Now we need to access this data. For this we may use sunpy:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import sunpy.map
import glob  # To fetch all files from a directory

map_list = sorted(glob.glob('LASCO_data/*.fts'))

lasco_map = sunpy.map.Map(map_list[0])
    
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=lasco_map)
lasco_map.plot(axes=ax, cmap = plt.get_cmap('soholasco3'))
lasco_map.draw_limb()
plt.show()

In [ ]:
lasco_map #get the time of the map

As we have just seen, it is fairly straightforward to get images from a spacecraft into Python and show them, but what can we do once we have these images?

## Making videos out of our material

A first good use of these images is to study the dynamics of the solar atmosphere, known to affect spacecraft and the Earth's weather! To do that, we need to have a look at all of the images in rapid succession.

In [ ]:
!pip install -q opencv-python

In [ ]:
import cv2 # Here, we import OpenCV, a powerful image and video processing tool
from os import getcwd
import glob
from shutil import rmtree
from os import makedirs as newdir
from os import path

if path.exists('WEW_PythonWorkshop_Database/'): # You wouldn't need these two lines if you're plotting a different date!
    rmtree(path.join('WEW_PythonWorkshop_Database/'))

! git clone https://github.com/andyto1234/WEW_PythonWorkshop_Database.git # We're downloading data from github so it's quicker!

dir_png = 'WEW_PythonWorkshop_Database/LASCO_images'
img_array = []

for filename in sorted(glob.glob(f'{dir_png}/*.png')):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('solar_vid_coronograph.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 5, size)

for i in range(len(img_array)):
    out.write(img_array[i])

out.release()

With the above, we will have created a movie called "solar_vid_coronograph.mp4", which holds all of our images together.